# Harmonization across studies with PIC-SURE

This tutorial notebook will demonstrate how to query and work with the BioData Catalyst studies, particularly cross-study harmonization. For a more step-by-step introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to review the \"Get your security token\" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

 -------   

# Environment set-up

### System requirements
- Python 3.6 or later
- pip python package manager, already available in most systems with a python interpreter installed


### Install packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git@new-search

import PicSureBdcAdapter

## Connecting to a PIC-SURE network

In [ ]:
# Uncomment production URL when testing in production
# PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)

 -------   

## Harmonizing variables with PIC-SURE
One of the key challenges to conducting analyses with several studies is ensuring correct data harmonization, or combining of data from different sources. There are many harmonization techniques, but this notebook will demonstrate how to find and extract similar variables from different studies in PIC-SURE. Two examples of this will be shown:
1. Retrieving variables for *sex and gender* across studies with BMI
2. Harmonizing the variable *"orthopnea"* across studies with age


*For more information about the TOPMed DCC Harmonized Data Set in PIC-SURE, please refer to the [`2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb`](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/blob/master/NHLBI_BioData_Catalyst/python/2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb) notebook*

-----

## Sex and gender variables across studies
<font color='darkgreen'>**Goal: Create harmonized variables for sex and BMI which combine data from multiple studies**</font> 

These variables are labelled differently for each of these studies. For example, some use the keyword `sex` while others use `gender`. To acccount for these differences, we need to develop a way to search for multiple keywords at once.

Let's start by searching for `sex` and `gender` to gain a better understanding of the variables that exist in PIC-SURE with these terms.



In [ ]:
sex_dictionary = bdc.useDictionary().dictionary().find('sex|gender')
sex_dataframe = sex_dictionary.dataframe()
print(sex_dataframe.shape)
sex_dataframe.head()


After reviewing the variables using the dataframe (or the [user interface](https://picsure.biodatacatalyst.nhlbi.nih.gov/psamaui/login)), let's say we are interested in sex/gender variables from the following studies:
- ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints) (phs001252)
- EOCOPD (Early Onset of COPD) (phs000946)

We can find the study IDs in the data access dashboard in the user interface.

We have already used the PIC-SURE API dictionary object and find method to find variables which contain the keywords sex and gender. Let's filter these results to our desired studies of interest.

### Which sex variable in ECLIPSE should we use? 
We can see that there are many sex/gender variables in ECLIPSE originating from various data tables.

We will examine the data associated with these variables to determine which one to use for our analysis.

In [ ]:
# which sex/gender variables are part of ECLIPSE?
eclipse_vars = sex_dataframe['studyId'].str.contains('phs001252')

# create a subset of only ECLIPSE sex/gender vars
eclipse_sex_df = sex_dataframe[eclipse_vars][['studyId', 'dataTableName', 'name', 'description', 'values', 'HPDS_PATH']]
eclipse_sex_df


In [ ]:
# Initialize a query
authPicSure = bdc.useAuthPicSure()
eclipse_sex_query = authPicSure.query()
eclipse_sex_query.anyof().add(eclipse_sex_df['HPDS_PATH'])
eclipse_sex_results = eclipse_sex_query.getResultsDataFrame()
eclipse_sex_results.head()

In [ ]:
# first let's see which sex/gender variables are the most complete for our dataset by counting the number of NA values
print(eclipse_sex_results.isna().sum())

# let's focus on those variables which do not have any NA values
filtered_eclipse_sex_results = eclipse_sex_results.loc[:, eclipse_sex_results.isna().sum() == 0]
filtered_eclipse_sex_results

In [ ]:
# in this example, we notice that all our filtered variables have equivalent data for each patient:
filtered_eclipse_sex_results.iloc[:,4:].nunique(axis = 1).unique()

In [ ]:
# so we select the first variable to use in our analysis.

# we can learn more about a variables by using the varInfo method
eclipse_sex_var = filtered_eclipse_sex_results.columns[5]
sex_dictionary.varInfo(eclipse_sex_var)

### Which sex variable in EOCOPD should we use? 
Unlike the ECLIPSE data, we see that there is only one sex variable in EOCOPD. It is called 'GENDER'.

We will save the HPDS_PATH associated to this variable so we can add it to our query later.

In [ ]:
# which sex/gender variables are part of EOCOPD?
eocopd_vars = sex_dataframe['studyId'].str.contains('phs000946')

# create a subset of only EOCOPD sex/gender vars
eocopd_sex_df = sex_dataframe[eocopd_vars][['studyId', 'dataTableName', 'name', 'description', 'values', 'HPDS_PATH']]
eocopd_sex_df

eocopd_sex_var = eocopd_sex_df[['HPDS_PATH']].iloc[0,0]
eocopd_sex_var

### Which BMI variables should we use?

We will follow the approach outlined above to first search our data dictionary for all variables containing the terms 'body mass index'. We then filter to only those BMI variables which are part of ECLIPSE or EOCOPD and view the variables with their associated metadata. By examining metadata like the dataTableName, description, nad name, we are able to determine which variables we are interested in:
- Body Mass Index [BMI ] for eocopd (\phs000946\pht005719\phv00266123\BMI\)
- Body mass index (kg/m2) from the bodycomp table for eclipse (\phs001252\pht006373\phv00293610\VSBMI\) 

In [ ]:
bmi_dictionary = bdc.useDictionary().dictionary().find('body mass index')
bmi_dataframe = bmi_dictionary.dataframe()

# which bmi variables are part of ECLIPSE or EOCOPD?
bmi_vars = bmi_dataframe['studyId'].str.contains('phs001252|phs000946')

# create a subset of only the ECLIPSE / EOCOPD BMI vars
bmi_dataframe = bmi_dataframe[bmi_vars][['studyId', 'dataTableName', 'name', 'description', 'values', 'HPDS_PATH']]
bmi_dataframe

# save HPDS_PATHs of interest
eocopd_bmi_var = "\phs000946\pht005719\phv00266123\BMI\\"
eclipse_bmi_var = "\phs001252\pht006373\phv00293610\VSBMI\\"

bmi_dataframe.head()

In [ ]:
combined_query = authPicSure.query()

# Build query using selected concept paths
combined_query.anyof().add([eclipse_sex_var, eclipse_bmi_var, eocopd_sex_var, eocopd_bmi_var])

In [ ]:
# Get query results
combined_results = combined_query.getResultsDataFrame()
combined_results.head()

In [ ]:
# rename columns
combined_results.columns = ['patient_id', 'parent_accession', 'topmed_accession', 'consent', 'eocopd_sex', 'eocopd_bmi', 'eclipse_bmi', 'eclipse_sex']
combined_results.head()


In [ ]:
# convert consent code to study name
combined_results['study'] = np.where(combined_results['consent'].str.contains('phs001252'), 'eclipse', 'eocopd')
combined_results.head()


In [ ]:
# combine sex variables
combined_results['sex'] = combined_results['eocopd_sex'].fillna(combined_results['eclipse_sex'])
combined_results.head()


In [ ]:
# combine bmi variables
combined_results['bmi'] = combined_results['eocopd_bmi'].fillna(combined_results['eclipse_bmi'])
combined_results.head()


In [ ]:
# remove unneccessary columns and summarize
plotdf = combined_results[[ 'study', 'sex', 'bmi']]
plotdf = plotdf.groupby(['study', 'sex']).mean().unstack()
plotdf

In [ ]:
plotdf.plot(kind = 'bar')


-----

## Orthopnea variables across studies
In this example, we will be harmonizing variables related to `orthopnea`. Orthopnea is shortness of breath that occurs when individuals lie flat. Because of this, people with orthopnea have to sleep propped up in bed or sitting in a chair. You can read more about this condition [here](https://www.sleepfoundation.org/sleep-apnea/orthopnea).

Let's start by doing a search in PIC-SURE for concept paths containing `orthopnea`.

### Search and find orthopnea variables

In [ ]:
orthopnea_dictionary = bdc.useDictionary().dictionary().find("orthopnea")
orthopnea_dataframe = orthopnea_dictionary.dataframe()
print(orthopnea_dataframe.shape)
orthopnea_dataframe[['name', 'description', 'values', 'studyId']].head()

As shown in the resulting dataframe, orthopnea is often recorded as a something like `sleep on two or more pillows to help you breathe?`

Let's do a search for `pillows` in PIC-SURE to explore those concept paths.

In [ ]:
pillows_dictionary = bdc.useDictionary().dictionary().find("pillows")
pillows_dataframe = pillows_dictionary.dataframe()
pillows_dataframe[['name', 'description', 'values', 'studyId']].head()

As shown in the resulting dataframe, there are some concept paths related to orthopnea that were not captured by the previous `orthopnea` search, such as `Do you sleep on 2 or more pillows to improve your breathing?` from the Jackson Heart Study (JHS) Cohort (phs000286). In fact, the JHS dataset was not returned at all when we searched for `orthopnea`. 

This is problematic for researchers studying these types of variables, since the concept paths of interest may not always be captured by a single search term. To account for this, researchers must conduct exploratory searches to determine potential search terms.

Let's say we want to harmonize orthopnea variables from the following datasets: 
- FHS (phs000007)
- MESA (phs000209)
- WHI (phs000200)

We can first get all concept paths related to our terms of interest (`orthopnea` or `pillows`) and then filter to our studies of interest.

You can get the phs number associated with each study from the data access dashboard.

In [ ]:
# search pic-sure for 'orthopnea' and 'pillows'
harmonized_dictionary = bdc.useDictionary().dictionary().find("orthopnea|pillows")
harmonized_df = harmonized_dictionary.dataframe()

# filter to variables found in FHS, MESA, WHI
harmonized_df = harmonized_df[harmonized_df['studyId'].str.contains('phs000007|phs000209|phs000200')]
print(harmonized_df.shape)
harmonized_df[['name', 'description', 'values', 'studyId']]

After browsing the available variables, we decided to choose the following:

In [ ]:
orthopnea_variables_of_interest = harmonized_df[harmonized_df['description'].isin(['Orthopnea', 'F136 Orthopnea', 'SLEEP ON 2+ PILLOWS TO HELP BREATHING'])]

# save variable paths for querying data
orthopnea_variable_paths_of_interest = orthopnea_variables_of_interest['HPDS_PATH'].tolist()

orthopnea_variables_of_interest[['values', 'studyId', 'description', 'name']]


### Search and find pneumonia variables

As part of our research, let's say we are interested in exploring the relationship between pneumonia and orthopnea. Let's save concept paths related to `pneumonia` as well. 

In [ ]:
# search pic-sure for 'pneumonia'
harmonized_dictionary = bdc.useDictionary().dictionary().find("pneumonia")
harmonized_df = harmonized_dictionary.dataframe()

# filter to variables found in FHS, MESA, WHI
harmonized_df = harmonized_df[harmonized_df['studyId'].str.contains('phs000007|phs000209|phs000200')]
print(harmonized_df.shape)
harmonized_df[['name', 'description', 'values', 'studyId']]

After browsing the available variables, we decided to choose the following:

In [ ]:
pneumonia_variables_of_interest = harmonized_df[harmonized_df['varId'].isin(['phv00255144.v1', 
                                                                             'phv00083539.v1', 
                                                                             'phv00283208.v1'])]

# save variable paths for querying data
pneumonia_variable_paths_of_interest = pneumonia_variables_of_interest['HPDS_PATH'].tolist()

pneumonia_variables_of_interest[['values', 'studyId', 'description', 'name', 'varId']]


### Make query

In [ ]:
# Initialize a query
authPicSure = bdc.useAuthPicSure()
orthopnea_query = authPicSure.query()
orthopnea_query.anyof().add(orthopnea_variable_paths_of_interest + pneumonia_variable_paths_of_interest)
orthopnea_results = orthopnea_query.getResultsDataFrame()
orthopnea_results.head()

In [ ]:
# rename columns
orthopnea_results.columns = ['patient_id', 'parent_accession', 'topmed_accession', 'consent', 
                             'fhs_pneumonia', 'fhs_orthopnea', 'whi_pneumonia', 
                             'whi_orthopnea', 'mesa_pneumonia', 'mesa_orthopnea']
orthopnea_results.head()


In [ ]:
# convert consent code to study name
orthopnea_results['study'] = np.where(orthopnea_results['consent'].str.contains('phs000007'), 'fhs', 
                                      np.where(orthopnea_results['consent'].str.contains('phs000209'), 'mesa', 'whi'))
orthopnea_results.head()


In [ ]:
# combine orthopnea variables
orthopnea_results['orthopnea'] = orthopnea_results['fhs_orthopnea'].fillna(orthopnea_results['whi_orthopnea'])
orthopnea_results['orthopnea'] = orthopnea_results['orthopnea'].fillna(orthopnea_results['mesa_orthopnea'])
orthopnea_results.head()

orthopnea_results['orthopnea'].unique()

---

In [ ]:
mesa_pneu = find_concept_paths('Multi-Ethnic Study of Atherosclerosis (MESA) SHARe ( phs000209 )', ['pneumonia'])
#mesa_pneu # Uncomment to view full results list
mesa_pneu_var = mesa_pneu[4]
mesa_pneu_var

In [ ]:
whi_pneu = find_concept_paths("Women's Health Initiative Clinical Trial and Observational Study ( phs000200 )", ['pneumonia'])
# whi_pneu # Uncomment to view full results list
whi_pneu_var = whi_pneu[0]
whi_pneu_var

In [ ]:
fhs_pneu = find_concept_paths("Framingham Cohort ( phs000007 )", ['pneumonia'])
# fhs_pneu # Uncomment to view full results
fhs_pneu_var = fhs_pneu[26]
fhs_pneu_var

Now that we know and have saved our concept paths of interest, we can use these to build our queries and obtain our dataframes. 

In [ ]:
mesa_query = resource.query()
mesa_query.anyof().add(mesa_var)
mesa_query.require().add(mesa_pneu_var)
mesa_results = mesa_query.getResultsDataFrame(low_memory=False)

In [ ]:
fhs_query = resource.query()
fhs_query.anyof().add(fhs_var)
fhs_query.require().add(fhs_pneu_var)
fhs_results = fhs_query.getResultsDataFrame(low_memory=False)

In [ ]:
whi_query = resource.query()
whi_query.anyof().add(whi_var)
whi_query.require().add(whi_pneu_var)
whi_results = whi_query.getResultsDataFrame(low_memory=False)

Now that we have our patient-level dataframes, we can combine them into a single, cohesive dataframe.

The following function accomplishes three main tasks:
1. Removes extra columns, such as consent information
2. Renames the Pneumonia and Orthopnea columns
3. Adds the Dataset column, which corresponds to the study

In [ ]:
def clean_up_df2(df, study):
    columns_to_drop = ['\\_Parent Study Accession with Subject ID\\', '\\_Topmed Study Accession with Subject ID\\', '\\_consents\\', '\\_harmonized_consent\\']
    df1 = df.drop(columns=columns_to_drop, errors='ignore')
    if 'pneumonia' in df1.columns.values[1].lower():
        df1.columns = ['Patient ID', 'Pneumonia', 'Orthopnea']
    else:
        df1.columns = ['Patient ID', 'Orthopnea', 'Pneumonia']
    df1['Dataset'] = study
    return df1

In [ ]:
clean_fhs = clean_up_df2(fhs_results, 'FHS')
clean_whi = clean_up_df2(whi_results, 'WHI')
clean_mesa = clean_up_df2(mesa_results, 'MESA')

As you may have noticed, the orthopnea and pneumonia variables are encoded differently between these studies. Specifically:
- In FHS, pneumonia is recorded as "NO", "MAYBE", or "YES"
- In MESA, pneumonia is recorded as "Yes", "No", or "Don't know / Not sure"
- In WHI, pneumonia is recoded as a value, or the number of times a participant has been diagnosed with pneumonia

To harmonize these variables, we need to create a consistent encoding across these studies. 

In [ ]:
# Combine the results for the pneumonia and orthopnea variables
pneumonia_results = list(clean_fhs['Pneumonia'].unique())+list(clean_whi['Pneumonia'].unique())+list(clean_mesa['Pneumonia'].unique())
orthopnea_results = list(clean_fhs['Orthopnea'].unique())+list(clean_whi['Orthopnea'].unique())+list(clean_mesa['Orthopnea'].unique())

#pneumonia_results # Uncomment to view values
#orthopnea_results # Uncomment to view values

The following code creates these mappings:

|Raw value|Harmonized value|
|---|---|
|No|No|
|NO|No|
|No/Not recorded|No|
|Numeric value equal to 0|No|
|MAYBE|Maybe|
|Don't know / Not sure|Do not know|
|DO NOT KNOW|Do not know|
|YES|Yes|
|Yes|Yes|
|Numeric value greater than 0|Yes|

In [ ]:
mappings = {}
for res in [pneumonia_results, orthopnea_results]:
    for val in res:
        if val not in mappings.keys():
            if isinstance(val, float):
                if val == 0.0:
                    mappings[val] = 'No'
                else:
                    mappings[val] = 'Yes'
            else:
                if val.lower() == 'yes':
                    mappings[val] = 'Yes'
                if 'know' in val.lower():
                    mappings[val] = 'Do not know'
                elif 'no' in val.lower():
                    mappings[val] = 'No'
                if 'maybe' in val.lower():
                    mappings[val] = 'Maybe'

final_df = pd.concat([clean for clean in [clean_fhs, clean_whi, clean_mesa]], ignore_index=True)
final_df = final_df.replace({"Pneumonia": mappings, 'Orthopnea':mappings})
final_df.head()

Now that the variables have been encoded in the same way, we can use this dataframe for analysis and visualizations. 

In [ ]:
# library
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

In [ ]:
# Filters to apply to the final_df to create figures
pneu_yes = final_df['Pneumonia'] == 'Yes'
pneu_no = final_df['Pneumonia'] == 'No'
ortho_yes = final_df['Orthopnea'] == 'Yes'
ortho_no = final_df['Orthopnea'] == 'No'
fhs = final_df['Dataset'] == 'FHS'
mesa = final_df['Dataset'] == 'MESA'
whi = final_df['Dataset'] == 'WHI'

In [ ]:
# FHS figure
pneu_count =  final_df[pneu_yes & ortho_no & fhs].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes & fhs].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes & fhs].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("FHS Dataset")
plt.show()

# MESA figure
pneu_count =  final_df[pneu_yes & ortho_no & mesa].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes & mesa].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes & mesa].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("MESA Dataset")
plt.show()

# WHI figure
pneu_count =  final_df[pneu_yes & ortho_no & whi].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes & whi].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes & whi].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("WHI Dataset")
plt.show()

# Combined figure
pneu_count =  final_df[pneu_yes & ortho_no].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("Harmonized")
plt.show()